In [25]:
# 🧩 Schritt 1: Imports und Setup
import spacy
from spacy.tokens import DocBin
from spacy.training.example import Example
import json
import random
from pathlib import Path
from spacy.util import minibatch
!python -m spacy download de_core_news_md
!pip install spacy-lookups-data



# 📁 Schritt 2: Funktion zum Laden von Daten aus JSON
def load_data_from_json(path):
    with open(path, "r", encoding="utf-8") as f:
        raw_data = json.load(f)
    if isinstance(raw_data, dict):
        raw_data = [raw_data]

    TRAIN_DATA = []
    for entry in raw_data:
        text = entry["text"]
        entities = [(label["start"], label["end"], label["label"]) for label in entry["labels"]]
        TRAIN_DATA.append((text, {"entities": entities}))
    return TRAIN_DATA

# 🔄 Lade Trainings- und Dev-Daten separat
train_data = load_data_from_json("../../../data/original/ground_truth_split/train_norm.json")
dev_data = load_data_from_json("../../../data/original/ground_truth_split/validation_norm.json")
print(f"📥 Trainingsbeispiele: {len(train_data)}, Dev-Beispiele: {len(dev_data)}")

# 🧠 Schritt 3: Lade spaCy-Basismodell
base_model = "de_core_news_md"
nlp = spacy.load(base_model)


# Stelle sicher, dass NER-Komponente existiert
if "ner" not in nlp.pipe_names:
    ner = nlp.add_pipe("ner", last=True)
else:
    ner = nlp.get_pipe("ner")

# Registriere alle Labels aus beiden Datensätzen
for dataset in (train_data, dev_data):
    for _, annotations in dataset:
        for start, end, label in annotations["entities"]:
            ner.add_label(label)

# 🚀 Schritt 4: Modell-Initialisierung mit allen Daten (nur für Labels!)
def get_examples():
    for text, ann in train_data + dev_data:
        yield Example.from_dict(nlp.make_doc(text), ann)

optimizer = nlp.resume_training()


# 🏋️ Schritt 5: Training (nur auf Trainingsdaten)
n_iter = 20
for i in range(n_iter):
    random.shuffle(train_data)
    losses = {}

    batches = minibatch(train_data, size=8)
    for batch in batches:
        examples = [Example.from_dict(nlp.make_doc(text), ann) for text, ann in batch]
        nlp.update(examples, drop=0.35, losses=losses)

    print(f"🔁 Iteration {i+1}/{n_iter}, Loss: {losses['ner']:.4f}")

# 💾 Schritt 6: Modell speichern
output_dir = Path("custom_spacy_model_doccano_labeling")
output_dir.mkdir(exist_ok=True)
nlp.to_disk(output_dir)
print(f"\n✅ Modell gespeichert unter: {output_dir.resolve()}")

# 🔍 Schritt 7: Modell laden und auf dev_data testen
nlp2 = spacy.load(output_dir)

print("\n📊 Evaluation auf dev_data:")
for text, _ in random.sample(dev_data, min(5, len(dev_data))):  # max. 5 Beispiele
    doc = nlp2(text)
    print(f"\n> {text}")
    for ent in doc.ents:
        print(f"  - {ent.text} ({ent.label_})")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 MB 23.4 MB/s eta 0:00:00a 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_md')
📥 Trainingsbeispiele: 96, Dev-Beispiele: 25
🔁 Iteration 1/20, Loss: 1196.5320
🔁 Iteration 2/20, Loss: 984.1071
🔁 Iteration 3/20, Loss: 908.2000
🔁 Iteration 4/20, Loss: 742.9484
🔁 Iteration 5/20, Loss: 567.1023
🔁 Iteration 6/20, Loss: 518.2200
🔁 Iteration 7/20, Loss: 420.0266
🔁 Iteration 8/20, Loss: 381.5320
🔁 Iteration 9/20, Loss: 348.7346
🔁 Iteration 10/20, Loss: 387.7892
🔁 Iteration 11/20, Loss: 420.1666
🔁 Iteration 12/20, Loss: 321.8346
🔁 Iteration 13/20, Loss: 403.2117
🔁 Iteration 14/20, Loss: 364.0188
🔁 Iteration 15/20, Loss: 314.1981
🔁 Iteration 16/20, Loss: 259.7162
🔁 Iteration 17/20, Loss: 253.7685
🔁 Iteration 18/20, Loss: 272.9169
🔁 Iteration 19/20, Loss: 310.0474
🔁 Iteration 20/20, Loss: 230.0445

✅ Modell gespeichert unter: /Users/timonmartens/Library/CloudStorage/OneDrive-Persönlich/D

In [22]:
# 🔍 Schritt 7: Modell laden
nlp2 = spacy.load(output_dir)

# 🧩 Schritt 7.1: EntityRuler hinzufügen
ruler = nlp2.add_pipe("entity_ruler", before="ner")  # wichtig: vor dem ner!
# Beispiel: Lade Muster
# Funktion zum Laden von Namen aus Datei
def load_names(path, label):
    with open(path, "r", encoding="utf-8") as f:
          names = [name.strip() for name in f if name.strip()]
          patterns = [{"label": label, "pattern": name} for name in names]
    return patterns, names

# Gazetteer laden
vornamen_patterns, vornamen_liste = load_names("Vornamen.txt", "VORNAME")
nachnamen_patterns, nachnamen_liste = load_names("Nachnamen.txt", "NACHNAME")
titel_patterns, titel_liste = load_names("Titel.txt", "TITEL")
wohnort_patterns, wohnort_liste = load_names("Orte.txt", "WOHNORT")
postleitzahl_patterns, postleitzahl_liste = load_names("Postleitzahlen.txt", "POSTLEITZAHL")
strasse_patterns = [
    {
        "label": "STRASSE",
        "pattern": [
            {"TEXT": {"REGEX": r".*(straße|gasse|allee|weg|platz|str.|grund)$"}},
            {"TEXT": {"REGEX": r"^\d+[a-zA-Z]?$"}}
        ]
    }
]
vertragsnummer_patterns = [
    {
        "label": "VERTRAGSNUMMER",
        "pattern": [
            {"LOWER": {"IN": ["vertragsnummer", "vertragsnr.", "vnr", "vn"]}},
            {"IS_PUNCT": True, "OP": "*"},
            {"TEXT": {"REGEX": r"^\d{6,12}\.?$"}}

        ]
    }
]

kundennummer_patterns = [
    {
        "label": "KUNDENNUMMER",
        "pattern": [
            {"LOWER": {"IN": ["kundennummer", "kundennr.", "kdnr", "kd"]}},
            {"IS_PUNCT": True, "OP": "*"},
            {"TEXT": {"REGEX": r"^\d{6,12}\.?$"}}
        ]
    }
]

zuordnungsnummer_patterns = [
    {
        "label": "ZUORDNUNGSNUMMER",
        "pattern": [
            {"LOWER": {"IN": ["znr", "zuordnungsnummer"]}},
            {"IS_PUNCT": True, "OP": "*"},
            {"TEXT": {"REGEX": r"^\d{6,12}\.?$"}}
        ]
    }
]
iban_pattern = [
    {"label": "IBAN", "pattern": [{"TEXT": {"REGEX": r"^[A-Z]{2}[0-9]{2}[A-Z0-9]{11,30}$"}}]}
]

bic_pattern = [
    {"label": "BIC", "pattern": [{"TEXT": {"REGEX": r"^[A-Z]{6}[A-Z2-9][A-NP-Z0-9]([A-Z0-9]{3})?$"}}]}
]

zahlung_pattern = [
    {
        "label": "ZAHLUNG",
        "pattern": [
            {"TEXT": {"REGEX": r"^\d+[.,]?\d{0,2}$"}},
            {"TEXT": {"REGEX": r"^(€|euro|eur)$"}}
        ]
    }
]

zählerstand_patterns = [
    {
        "label": "ZÄHLERSTAND",
        "pattern": [
            {"LOWER": {"IN": ["zählerstand"]}},
            {"IS_PUNCT": True, "OP": "*"},
            {"TEXT": {"REGEX": r"^\d+(\.\d+)?$"}}
        ]
    }
]

zählernummer_patterns = [
    {
        "label": "ZÄHLERNUMMER",
        "pattern": [
            {"LOWER": {"IN": ["zählernummer"]}},
            {"IS_PUNCT": True, "OP": "*"},
            {"TEXT": {"REGEX": r"^[A-Z0-9]{6,20}$"}}  # Groß- und Kleinbuchstaben + Ziffern erlaubt
        ]
    }
]

verbrauch_patterns = [
    {
        "label": "VERBRAUCH",
        "pattern": [
            {"LOWER": {"IN": ["verbrauch"]}},
            {"IS_PUNCT": True, "OP": "*"},
            {"TEXT": {"REGEX": r"^\d+(\.\d+)?$"}},
            {"LOWER": {"IN": ["kwh", "m³", "kw"]}, "OP": "?"}
        ]
    }
]

verbrauch_patterns += [
    {
        "label": "VERBRAUCH",
        "pattern": [
            {"LOWER": {"IN": ["verbrauch"]}},
            {"IS_PUNCT": True, "OP": "*"},
            {"TEXT": {"REGEX": r"^\d+(?:[.,]\d+)?(kwh|m³|kw)$"}}
        ]
    }
]


wlv_patterns = [
    {
        "label": "WLV",
        "pattern": [
            {"LOWER": {"IN": ["wlv"]}},
            {"IS_PUNCT": True, "OP": "*"},
            {"TEXT": {"REGEX": r"^\d{4,12}$"}}
        ]
    }
]

email_pattern = [
    {
        "label": "EMAIL",
        "pattern": [
            {"TEXT": {"REGEX": r"^[\w\.-]+@[\w\.-]+\.\w{2,}$"}}
        ]
    }
]

telefon_pattern = [
    {
        "label": "TELEFON",
        "pattern": [
            {"TEXT": {"REGEX": r"^(\+49|0)[\d\s/-]{7,}$"}}
        ]
    }
]

url_pattern = [
    {
        "label": "URL",
        "pattern": [
            {"TEXT": {"REGEX": r"^https?://[\w\-\.]+\.\w{2,}(/[\w\-\.]*)*$"}}
        ]
    },
    {
        "label": "URL",
        "pattern": [
            {"TEXT": {"REGEX": r"^www\.[\w\-\.]+\.\w{2,}(/[\w\-\.]*)*$"}}
        ]
    }
]

datum_pattern = [
    {
        "label": "DATUM",
        "pattern": [
            {"TEXT": {"REGEX": r"^(\d{1,2}[./-]){2}\d{2,4}$"}}  # z. B. 15.06.2024
        ]
    },
    {
        "label": "DATUM",
        "pattern": [
            {"TEXT": {"REGEX": r"^\d{4}-\d{2}-\d{2}$"}}  # z. B. 2024-06-15
        ]
    },
    {
        "label": "DATUM",
        "pattern": [
            {"TEXT": {"REGEX": r"^\d{1,2}$"}},  # z. B. 15
            {"LOWER": {"IN": [
                "januar", "jan", "februar", "feb", "märz", "maerz", "mrz", "april", "apr",
                "mai", "juni", "jun", "juli", "jul", "august", "aug", "september", "sep",
                "oktober", "okt", "november", "nov", "dezember", "dez"
            ]}},
            {"TEXT": {"REGEX": r"^\d{2,4}$"}, "OP": "?"}  # optional Jahr
        ]
    }
]





# EntityRuler erstellen und Muster hinzufügen
ruler = nlp2.get_pipe("entity_ruler")
ruler.add_patterns(iban_pattern + bic_pattern + zahlung_pattern + zählerstand_patterns + email_pattern + telefon_pattern)  # 👈 Muster hinzufügen!


# 💾 (Optional) Modell MIT Ruler neu speichern
output_dir_ruler = Path("custom_spacy_model_with_ruler")
output_dir_ruler.mkdir(exist_ok=True)
nlp2.to_disk(output_dir_ruler)
print(f"✅ Modell mit EntityRuler gespeichert unter: {output_dir_ruler.resolve()}")

✅ Modell mit EntityRuler gespeichert unter: /Users/timonmartens/Library/CloudStorage/OneDrive-Persönlich/Desktop/Veranstaltungen/Data Analytics in Applications/daia-eon/notebooks/3_model_training_and_testing/spacy_pipeline/custom_spacy_model_with_ruler


In [23]:
from spacy.training import Example
from spacy.scorer import Scorer
import pandas as pd
from collections import defaultdict

def evaluate_model_with_counts(nlp, examples):
    scorer = Scorer()
    example_list = []

    # Vorbereitung: TP/FP/FN zählen
    counts = defaultdict(lambda: {"tp": 0, "fp": 0, "fn": 0})

    for text, ann in examples:
        doc = nlp(text)
        example = Example.from_dict(doc, ann)
        pred_ents = {(ent.start_char, ent.end_char, ent.label_) for ent in example.predicted.ents}
        true_ents = {(ent.start_char, ent.end_char, ent.label_) for ent in example.reference.ents}

        for ent in pred_ents:
            if ent in true_ents:
                counts[ent[2]]["tp"] += 1
            else:
                counts[ent[2]]["fp"] += 1
        for ent in true_ents:
            if ent not in pred_ents:
                counts[ent[2]]["fn"] += 1

        example_list.append(example)

    # Evaluation mit spaCy
    scores = scorer.score(example_list)
    results = []

    for label, metrics in scores["ents_per_type"].items():
        tp = counts[label]["tp"]
        fp = counts[label]["fp"]
        fn = counts[label]["fn"]

        results.append({
            "Label": label,
            "Precision (%)": round(metrics["p"] * 100, 2),
            "Recall (%)": round(metrics["r"] * 100, 2),
            "F1-Score (%)": round(metrics["f"] * 100, 2),
            "True Positives": tp,
            "False Positives": fp,
            "False Negatives": fn
        })

    return pd.DataFrame(results)

In [24]:
results_df = evaluate_model_with_counts(nlp2, dev_data)
display(results_df)

,Label,Precision (%),Recall (%),F1-Score (%),True Positives,False Positives,False Negatives
0,VORNAME,90.91,96.77,93.75,30,3,1
1,NACHNAME,89.47,89.47,89.47,34,4,4
2,TELEFONNUMMER,33.33,50.00,40.00,2,4,2
3,VERTRAGSNUMMER,72.22,92.86,81.25,13,5,1
4,TITEL,80.00,100.00,88.89,4,1,0
5,STRASSE,88.89,100.00,94.12,8,1,0
6,HAUSNUMMER,100.00,100.00,100.00,8,0,0
7,POSTLEITZAHL,100.00,100.00,100.00,8,0,0
8,WOHNORT,100.00,87.50,93.33,7,0,1
9,FIRMA,0.00,0.00,0.00,0,2,3
